In [2]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
from scipy.stats import randint as sp_randint
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv("final_v5_dataset.csv")
data.head()

,Unnamed: 0,time,ticker,abnormal_return,y,volatility,reaction_positive,reaction_negative,volume_adi,volume_mfi,...,item7_polarity,item7_subjectivity,industry_B,industry_C,industry_D,industry_E,industry_F,industry_G,industry_H,industry_I
0,0,2006,nwpx,0.635924,1,0.062822,5,6,-3.515938e+05,50.359009,...,0.519824,0.313428,0,0,1,0,0,0,0,0
1,1,2007,nwpx,0.026142,1,0.147784,5,6,-1.440436e+06,57.280553,...,0.487179,0.326236,0,0,1,0,0,0,0,0
2,2,2008,nwpx,-0.762775,0,0.180493,3,8,-1.414003e+06,53.844115,...,0.490050,0.307105,0,0,1,0,0,0,0,0
3,3,2010,nwpx,0.210231,1,0.075963,10,1,-4.214262e+06,50.947005,...,0.429119,0.321627,0,0,1,0,0,0,0,0
4,4,2011,nwpx,-0.013295,0,0.161627,6,5,-4.860406e+06,49.961210,...,0.500000,0.309386,0,0,1,0,0,0,0,0


# Without feature based on literature review

In [34]:
data_vol = data.drop(["volatility"],axis = 1)
data_vol.head()

,Unnamed: 0,time,ticker,abnormal_return,y,reaction_positive,reaction_negative,volume_adi,volume_mfi,volatility_atr,...,item7_polarity,item7_subjectivity,industry_B,industry_C,industry_D,industry_E,industry_F,industry_G,industry_H,industry_I
0,0,2006,nwpx,0.635924,1,5,6,-3.515938e+05,50.359009,0.792847,...,0.519824,0.313428,0,0,1,0,0,0,0,0
1,1,2007,nwpx,0.026142,1,5,6,-1.440436e+06,57.280553,1.412616,...,0.487179,0.326236,0,0,1,0,0,0,0,0
2,2,2008,nwpx,-0.762775,0,3,8,-1.414003e+06,53.844115,2.589620,...,0.490050,0.307105,0,0,1,0,0,0,0,0
3,3,2010,nwpx,0.210231,1,10,1,-4.214262e+06,50.947005,0.922209,...,0.429119,0.321627,0,0,1,0,0,0,0,0
4,4,2011,nwpx,-0.013295,0,6,5,-4.860406e+06,49.961210,1.204679,...,0.500000,0.309386,0,0,1,0,0,0,0,0


In [35]:
y=data_vol['y']
X=data_vol.iloc[:,5:]

In [36]:
# data processing for imbalanced data 
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from collections import Counter
from imblearn.over_sampling import SMOTE 
standardizer = StandardScaler()
sm = SMOTE(random_state=42) 
X_sm, y_sm = sm.fit_resample(X, y) 
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm,test_size = 0.2)
standardizer.fit(X_train) 
X_train = standardizer.transform(X_train) 
X_test = standardizer.transform(X_test)

In [37]:
xgb_classifier = xgb.XGBClassifier(gamma=0,n_estimators=260,max_depth=11,min_child_weight=1,eval_metric=['logloss','auc','error'],use_label_encoder=False,random_state = 88)
xgb_classifier.fit(X_train,y_train)
lgbm_classifier = lgb.LGBMClassifier(max_depth=7,reg_alpha=0, min_child_weight=1,random_state = 88, n_estimators=120)
lgbm_classifier.fit(X_train,y_train)
rf_classifier = RandomForestClassifier(max_features=5,min_samples_leaf=4,max_depth=12, n_estimators=200,random_state = 88)
rf_classifier.fit(X_train,y_train)
tree_classifier =  DecisionTreeClassifier(min_samples_leaf=10,max_depth=12,random_state = 88,splitter='random')
tree_classifier.fit(X_train,y_train)
gb_classifier = GradientBoostingClassifier(max_depth=12, n_estimators=260,random_state = 88)
gb_classifier.fit(X_train,y_train)

GradientBoostingClassifier(max_depth=12, n_estimators=260, random_state=88)

In [38]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy as np

folds = StratifiedKFold(n_splits = 5)
def get_oof(clf ,x_train, y_train, x_test, n_folds = 5):
    """K-fold stacking"""
    ntrain = X_train.shape[0]
    ntest =  X_test.shape[0]
    classnum = len(np.unique(y_train))
    oof_test_skf = np.empty((5, ntest))
    kf = KFold(n_splits=n_folds)
    oof_train = np.zeros((ntrain,classnum))
    oof_test = np.zeros((ntest,classnum))
        
    for i,(train_index,test_index) in enumerate(kf.split(X_train)):
        #print(type(train_index))
        
        kf_X_train = X_train[train_index] # 数据
        kf_y_train = y_train[train_index] # 标签
        
        kf_X_test = X_train[test_index]  # k-fold的验证集
        
        clf.fit(kf_X_train, kf_y_train)
        
        oof_train[test_index] = clf.predict_proba(kf_X_test)
        #oof_test_skf[i,:]=clf.predict_proba(X_test)[:,0]
        oof_test += clf.predict_proba(X_test)
    oof_test = oof_test/float(n_folds)
    return oof_train, oof_test

In [39]:
# new feature input level1
import functools 
newfeature_list_1 = []
newtestdata_list_1 = []
xgb_oof_train_1, xgb_oof_test_1 = get_oof(xgb_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(xgb_oof_train_1)
newtestdata_list_1.append(xgb_oof_test_1)

lgbm_oof_train_1, xgb_oof_test_1 = get_oof(lgbm_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(lgbm_oof_train_1)
newtestdata_list_1.append(xgb_oof_test_1)

rf_oof_train_1, rf_oof_test_1 = get_oof(rf_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(rf_oof_train_1)
newtestdata_list_1.append(rf_oof_test_1)

dt_oof_train_1, dt_oof_test_1 = get_oof(tree_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(dt_oof_train_1)
newtestdata_list_1.append(dt_oof_test_1)

gb_oof_train_1, gb_oof_test_1 = get_oof(gb_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(gb_oof_train_1)
newtestdata_list_1.append(gb_oof_test_1)



In [40]:
newfeature_1 = functools.reduce(lambda x,y:np.concatenate((x,y),axis=1),newfeature_list_1)    
newtestdata_1 =functools.reduce(lambda x,y:np.concatenate((x,y),axis=1),newtestdata_list_1)

In [41]:
lr = LogisticRegression()
lr.fit(newfeature_1, y_train)
pred_1 = lr.predict(newtestdata_1)
accuracy_1 = accuracy_score(y_test, pred_1)
print(accuracy_1)

0.7300884955752213


In [42]:
from sklearn.svm import SVC
svr = SVC(kernel = 'rbf',C=1, gamma = 'scale',probability=True)
svr.fit(newfeature_1, y_train)
pred_2 = svr.predict(newtestdata_1)
accuracy_2 = accuracy_score(y_test, pred_2)
print(accuracy_2)

0.7374631268436578


# Without feature based on dictionary approach

In [75]:
data_rea = data.drop(["reaction_positive","reaction_negative"],axis = 1)
data_rea.head()

,Unnamed: 0,time,ticker,abnormal_return,y,volatility,volume_adi,volume_mfi,volatility_atr,trend_sma_slow,...,item7_polarity,item7_subjectivity,industry_B,industry_C,industry_D,industry_E,industry_F,industry_G,industry_H,industry_I
0,0,2006,nwpx,0.635924,1,0.062822,-3.515938e+05,50.359009,0.792847,28.379537,...,0.519824,0.313428,0,0,1,0,0,0,0,0
1,1,2007,nwpx,0.026142,1,0.147784,-1.440436e+06,57.280553,1.412616,35.654050,...,0.487179,0.326236,0,0,1,0,0,0,0,0
2,2,2008,nwpx,-0.762775,0,0.180493,-1.414003e+06,53.844115,2.589620,43.180485,...,0.490050,0.307105,0,0,1,0,0,0,0,0
3,3,2010,nwpx,0.210231,1,0.075963,-4.214262e+06,50.947005,0.922209,21.281775,...,0.429119,0.321627,0,0,1,0,0,0,0,0
4,4,2011,nwpx,-0.013295,0,0.161627,-4.860406e+06,49.961210,1.204679,24.127654,...,0.500000,0.309386,0,0,1,0,0,0,0,0


In [76]:
y=data_rea['y']
X=data_rea.iloc[:,5:]

In [77]:
# data processing for imbalanced data 
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from collections import Counter
from imblearn.over_sampling import SMOTE 
standardizer = StandardScaler()
sm = SMOTE(random_state=42) 
X_sm, y_sm = sm.fit_resample(X, y) 
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm,test_size = 0.2)
standardizer.fit(X_train) 
X_train = standardizer.transform(X_train) 
X_test = standardizer.transform(X_test)

In [78]:
xgb_classifier = xgb.XGBClassifier(gamma=0,n_estimators=260,max_depth=11,min_child_weight=1,eval_metric=['logloss','auc','error'],use_label_encoder=False,random_state = 88)
xgb_classifier.fit(X_train,y_train)
lgbm_classifier = lgb.LGBMClassifier(max_depth=7,reg_alpha=0, min_child_weight=1,random_state = 88, n_estimators=120)
lgbm_classifier.fit(X_train,y_train)
rf_classifier = RandomForestClassifier(max_features=5,min_samples_leaf=4,max_depth=12, n_estimators=200,random_state = 88)
rf_classifier.fit(X_train,y_train)
tree_classifier =  DecisionTreeClassifier(min_samples_leaf=10,max_depth=12,random_state = 88,splitter='random')
tree_classifier.fit(X_train,y_train)
gb_classifier = GradientBoostingClassifier(max_depth=12, n_estimators=260,random_state = 88)
gb_classifier.fit(X_train,y_train)

GradientBoostingClassifier(max_depth=12, n_estimators=260, random_state=88)

In [79]:
# new feature input level1
import functools 
newfeature_list_1 = []
newtestdata_list_1 = []
xgb_oof_train_1, xgb_oof_test_1 = get_oof(xgb_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(xgb_oof_train_1)
newtestdata_list_1.append(xgb_oof_test_1)

lgbm_oof_train_1, xgb_oof_test_1 = get_oof(lgbm_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(lgbm_oof_train_1)
newtestdata_list_1.append(xgb_oof_test_1)

rf_oof_train_1, rf_oof_test_1 = get_oof(rf_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(rf_oof_train_1)
newtestdata_list_1.append(rf_oof_test_1)

dt_oof_train_1, dt_oof_test_1 = get_oof(tree_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(dt_oof_train_1)
newtestdata_list_1.append(dt_oof_test_1)

gb_oof_train_1, gb_oof_test_1 = get_oof(gb_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(gb_oof_train_1)
newtestdata_list_1.append(gb_oof_test_1)

In [80]:
newfeature_1 = functools.reduce(lambda x,y:np.concatenate((x,y),axis=1),newfeature_list_1)    
newtestdata_1 =functools.reduce(lambda x,y:np.concatenate((x,y),axis=1),newtestdata_list_1)

In [81]:
lr = LogisticRegression()
lr.fit(newfeature_1, y_train)
pred_1 = lr.predict(newtestdata_1)
accuracy_1 = accuracy_score(y_test, pred_1)
print(accuracy_1)

0.7514749262536873


In [82]:
from sklearn.svm import SVC
svr = SVC(kernel = 'rbf',C=1, gamma = 'scale',probability=True)
svr.fit(newfeature_1, y_train)
pred_2 = svr.predict(newtestdata_1)
accuracy_2 = accuracy_score(y_test, pred_2)
print(accuracy_2)

0.7617994100294986


# Without feature based on text classification approach

In [83]:
data_tca = data.drop(["industry_B","industry_C","industry_D","industry_E","industry_F","industry_G"
                     ,"industry_H","industry_I"],axis = 1)
data_tca.head()

,Unnamed: 0,time,ticker,abnormal_return,y,volatility,reaction_positive,reaction_negative,volume_adi,volume_mfi,...,risk_factor_len,item1_pos,item1_neg,item1_polarity,item1_subjectivity,item1_len,item7_pos,item7_neg,item7_polarity,item7_subjectivity
0,0,2006,nwpx,0.635924,1,0.062822,5,6,-3.515938e+05,50.359009,...,1789,438,174,0.431373,0.277174,2897,690,218,0.519824,0.313428
1,1,2007,nwpx,0.026142,1,0.147784,5,6,-1.440436e+06,57.280553,...,1731,441,178,0.424879,0.276957,2630,638,220,0.487179,0.326236
2,2,2008,nwpx,-0.762775,0,0.180493,3,8,-1.414003e+06,53.844115,...,1760,354,139,0.436105,0.268519,2618,599,205,0.490050,0.307105
3,3,2010,nwpx,0.210231,1,0.075963,10,1,-4.214262e+06,50.947005,...,3280,362,136,0.453815,0.260733,3246,746,298,0.429119,0.321627
4,4,2011,nwpx,-0.013295,0,0.161627,6,5,-4.860406e+06,49.961210,...,3232,388,142,0.464151,0.269857,3271,759,253,0.500000,0.309386


In [84]:
y=data_tca['y']
X=data_tca.iloc[:,5:]

In [85]:
# data processing for imbalanced data 
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from collections import Counter
from imblearn.over_sampling import SMOTE 
standardizer = StandardScaler()
sm = SMOTE(random_state=42) 
X_sm, y_sm = sm.fit_resample(X, y) 
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm,test_size = 0.2)
standardizer.fit(X_train) 
X_train = standardizer.transform(X_train) 
X_test = standardizer.transform(X_test)

In [86]:
xgb_classifier = xgb.XGBClassifier(gamma=0,n_estimators=260,max_depth=11,min_child_weight=1,eval_metric=['logloss','auc','error'],use_label_encoder=False,random_state = 88)
xgb_classifier.fit(X_train,y_train)
lgbm_classifier = lgb.LGBMClassifier(max_depth=7,reg_alpha=0, min_child_weight=1,random_state = 88, n_estimators=120)
lgbm_classifier.fit(X_train,y_train)
rf_classifier = RandomForestClassifier(max_features=5,min_samples_leaf=4,max_depth=12, n_estimators=200,random_state = 88)
rf_classifier.fit(X_train,y_train)
tree_classifier =  DecisionTreeClassifier(min_samples_leaf=10,max_depth=12,random_state = 88,splitter='random')
tree_classifier.fit(X_train,y_train)
gb_classifier = GradientBoostingClassifier(max_depth=12, n_estimators=260,random_state = 88)
gb_classifier.fit(X_train,y_train)

GradientBoostingClassifier(max_depth=12, n_estimators=260, random_state=88)

In [87]:
# new feature input level1
import functools 
newfeature_list_1 = []
newtestdata_list_1 = []
xgb_oof_train_1, xgb_oof_test_1 = get_oof(xgb_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(xgb_oof_train_1)
newtestdata_list_1.append(xgb_oof_test_1)

lgbm_oof_train_1, xgb_oof_test_1 = get_oof(lgbm_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(lgbm_oof_train_1)
newtestdata_list_1.append(xgb_oof_test_1)

rf_oof_train_1, rf_oof_test_1 = get_oof(rf_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(rf_oof_train_1)
newtestdata_list_1.append(rf_oof_test_1)

dt_oof_train_1, dt_oof_test_1 = get_oof(tree_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(dt_oof_train_1)
newtestdata_list_1.append(dt_oof_test_1)

gb_oof_train_1, gb_oof_test_1 = get_oof(gb_classifier,X_train, y_train.values, X_test)
newfeature_list_1.append(gb_oof_train_1)
newtestdata_list_1.append(gb_oof_test_1)

In [88]:
newfeature_1 = functools.reduce(lambda x,y:np.concatenate((x,y),axis=1),newfeature_list_1)    
newtestdata_1 =functools.reduce(lambda x,y:np.concatenate((x,y),axis=1),newtestdata_list_1)

In [89]:
lr = LogisticRegression()
lr.fit(newfeature_1, y_train)
pred_1 = lr.predict(newtestdata_1)
accuracy_1 = accuracy_score(y_test, pred_1)
print(accuracy_1)

0.7529498525073747


In [90]:
from sklearn.svm import SVC
svr = SVC(kernel = 'rbf',C=1, gamma = 'scale',probability=True)
svr.fit(newfeature_1, y_train)
pred_2 = svr.predict(newtestdata_1)
accuracy_2 = accuracy_score(y_test, pred_2)
print(accuracy_2)

0.7617994100294986
